In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dev_interaction_util import *
from DevCvAnalysis import DevCvAnalysis

In [3]:
config = load_config("config.yml") 

dropbox_data_dir = config['dropbox_data_dir']


The basic structure is the following:

1. Run the following cross-validated analyses:
   1. Predicting change by condition only
   2. Predicting change by condition and neural and behavioral measures
   3. Predicting change by condition, neural and behavioral measures, and their interactions
2. Measure the predictivity of the three models above using anova
3. Repeat the steps above separately for three outcome variables, change in: FFQ, ASA-24, and BFP
4. 

Tasks to do to get this job done (not in order):

1. Write the analysis pipeline above
2. Get the neural data
3. Get the behavioral data


We have the behavioral data. Do we have the neural data already?

What could we delegate here? Behavioral data we already have. We have mostly writen the analysis pipeline. The neural data could be passed on.

# Get the data

## Set up data


In [4]:
%load_ext autoreload
%autoreload 2

from InterventionCVManager import *

dropbox_data_dir = config['dropbox_data_dir']

icvm = InterventionCVManager(dropbox_data_dir)
icvm.mode = 'fast_pipeline_test'
icvm.group_mode = 'dichotomous'
#dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%load_ext autoreload
%autoreload 2
dev_cv_analysis = icvm.get_prepopulated_dev_cv_analysis(set_as_random=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/analyses/intervention_moderation/dev_interaction_util.py:918: FutureWarning: The default value of regex will change from True to False in a future version.
  ms_groups['intervention_group'] = ms_groups['group_raw'].str.replace(r"\(.*\)","")


pre merge: 275
post merge: 270
participants in pre merge but not post merge: {'DEV022', 'DEV280', 'DEV002', 'DEV032', 'DEV007'}
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'run', 'task'],
      dtype='object')
Index(['Unnamed: 0', 'subject_id', 'wave', 'spm_l2_path', 'condition',
       'beta_name', 'mask_label', 'roi_activity', 'task', 'run'],
      dtype='object')


# Predict change

### Set up sets of variables to run

First, we set up a function that loops runs the scoring loop above (which does one cross-validation analysis), and the nadditionally:
- selects the best model based on the overall results
- Runs a final fit
- presents model results

Now, we include functions that compare models with and without individual differences and interactions.

Should manually verify that in the following list, the intervention_group allocations are randomized (if we're running a test run) or that they are accurate (if it's not a test run).

In [6]:
pd.concat([
    dev_cv_analysis.outcome_measures['SID'],
      dev_cv_analysis.group_assignments
],axis=1)

,SID,intervention_group
0,DEV001,control
1,DEV004,control
2,DEV005,control
3,DEV008,control
4,DEV009,control
...,...,...
239,DEV308,control
240,DEV309,control
241,DEV310,control
242,DEV311,control


In [7]:
dev_cv_analysis.outcome_measures.columns

Index(['SID', 'cancer_promoting_minus_preventing_FFQ', 'cancer_promoting_FFQ',
       'bf', 'NUTRIENT_DENSITY_2wkAverage'],
      dtype='object')

In [8]:
condition_cols = dev_cv_analysis.group_assignment_onehots.columns.tolist()
inddiff_cols = dev_cv_analysis.get_predictors_main_names()

interaction_cols = [id + "*" + cond for id in inddiff_cols for cond in condition_cols]

predictor_sets = {
    'condition_inddiff_interactions': condition_cols + inddiff_cols + interaction_cols,
    'condition_inddiff': condition_cols + inddiff_cols,
    'condition_only': condition_cols
    
    
}


outcome_vars_to_try = [ 'bf','cancer_promoting_FFQ',
       'NUTRIENT_DENSITY_2wkAverage']

In [9]:
print(", ".join(dev_cv_analysis.get_predictor_data().columns))

BSCS, EDM, BIS_11, PCS, RS, TRSQ, ACES_sum, BFI_agreeableness, BFI_conscientiousness, BFI_extraversion, BFI_neuroticism, BFI_openness, IMI_effort_importance, IMI_interest_enjoyment, NCS_total, PLAN_cognitive_strategies, PLAN_mental_flexibility, PLAN_temporal_orientation, RMQ_assessment, TESQ_E_sum, SRHI_healthy_minus_unhealthy, RTFS_f1_minus_f2, cancer_promoting_minus_preventing_craved_FCI, cancer_promoting_minus_preventing_liked_FCI, cSES, age365, education_own, SST_SSD, SST_mean_ssrt_0, ROC_Crave_Regulate_Minus_Look, WTP_unhealthy_minus_healthy, wtp_liked_koban_kober_craving_wmapN99_boot10K_02-May-2022_notzero, wtp_liked_value_association-test_z_FDR_0.01, roc_reappraiseCrave_reappraisal_association-test_z_FDR_0.01, roc_reappraiseCrave_abstract_association-test_z_FDR_0.01, roc_reappraiseCrave_multivariate_regulation, roc_lookCrave_koban_kober_craving_wmapN99_boot10K_02-May-2022_notzero, sst_CorrectGo_striatum_joint_mask, sst_CorrectGo_finger movements_association-test_z_FDR_0.01, sst_

## Body Fat Percentage

In [10]:


model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, outcome_vars_to_try[0], dev_cv_analysis)

## condition_inddiff_interactions

loading raw data


UnboundLocalError: cannot access local variable 'devCVAnalysis' where it is not associated with a value

## Unhealthy foods

In [ ]:


model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'cancer_promoting_FFQ')

## Nutrient density

In [ ]:


model_outcomes = icvm.do_predictor_set_comparison(
    predictor_sets, 'NUTRIENT_DENSITY_2wkAverage')